## Let's see if we can wire this up to an openbb function
(Even if it's hardcoded for now)


- Agent
- Tools
- Prompt
- AgentExecutor (runtime)


In [1]:
import inspect

from openbb import obb
from openbb_core.app.router import CommandMap
from openbb_core.app.provider_interface import ProviderInterface

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool, Tool, AgentExecutor, AgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools import StructuredTool

In [3]:
from pydantic.v1 import create_model, BaseModel
from pydantic.v1.fields import FieldInfo
from pydantic_core import PydanticUndefinedType

Goal: let's make it easy to convert an OpenBB function --> Langchain / openai function.

Schema comes from the ProviderInterface  
Function / Endpoint map comes from CommandMap + scraping the Members

In [3]:
# All the magic lives in here
from utils import map_openbb_collection_to_langchain_tools
test_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")

In [181]:
# Let's add some memory
MEMORY_KEY = "chat_history"

# Create our prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're a powerful financial advisor.
            Do your best to answer the questions,
            and make use of the tools provided, paying special attention to the outputs they provide."""
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        (
            "user",
            "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Bind our tools
llm = ChatOpenAI(model="gpt-4-1106-preview")
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in compare_tools])  # Never forget!


# Create the agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=compare_tools, verbose=True)

In [183]:
type(agent)

langchain.schema.runnable.base.RunnableSequence

In [156]:
OpenAIFunctionsAgentOutputParser?

Init signature: OpenAIFunctionsAgentOutputParser() -> None
Docstring:     
Parses a message into agent action/finish.

Is meant to be used with OpenAI models, as it relies on the specific
function_call parameter from OpenAI to convey what tools to use.

If a function_call parameter is passed, then that is used to get
the tool and tool input.

If one is not passed, then the AIMessage is assumed to be the final output.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/miniforge3/envs/ds/lib/python3.10/site-packages/langchain/agents/output_parsers/openai_functions.py
Type:           ModelMetaclass
Subclasses:     

In [ ]:
chat_history = []

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

human_input = "List the competitors for TSLA. Use tools."

response = agent_executor.invoke(
    {
        "input": human_input,
        "chat_history": chat_history
    }
    
)

chat_history.extend(
    [
        HumanMessage(content=human_input),
        AIMessage(content=response["output"])
    ]
)



> Entering new AgentExecutor chain...

Invoking: `peers` with `{'symbol': 'TSLA'}`


peers_list=['XPEV', 'LI', 'RIVN', 'LCID', 'GM', 'NIO', 'F', 'FSR', 'MULN']The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's incre

In [ ]:
print(response['output'])

The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's increasing its investment in electric vehicles.
8. Fisker Inc. (FSR) - An American electric vehicle company that is developing electric passenger vehicles.
9. Mullen 

# Let's explore different agents

In [218]:
from langchain import hub
from langchain.agents import initialize_agent
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [222]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

# Let's partially fill out the prompt
tools = compare_tools
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)

llm_with_stop = llm.bind(stop=["\nObservation"])

In [227]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [228]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [230]:
response = agent_executor.invoke(
    {
        "input": (
            "Who are TSLA's competitors? "
        )
    }
)




> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-1CpZ1***************************************GVnE. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## Let's try and do tool retrieval

In [75]:
from langchain.agents import AgentExecutor, AgentOutputParser, LLMSingleActionAgent, Tool

from langchain.chains import LLMChain
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish

In [17]:
# We embed the tools using a vector store.
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [23]:
fundamental_tool_docs = [
    Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(fundamental_tools)
]

In [26]:
vector_store = FAISS.from_documents(fundamental_tool_docs, OpenAIEmbeddings())

In [50]:
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [fundamental_tools[d.metadata["index"]] for d in docs]
    return docs

In [102]:
# We have the tool retriever. Now we need to pair it with our agent.
# Let's start with the prompt

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give your answers in a bulleted list.

Question: {input}
{agent_scratchpad}""" 

In [208]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

system_template = """

"""

template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant"
        ),
        (
            "human", "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


In [169]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    input_variables=["input", "intermediate_steps"]
)

In [209]:
output_parser = OpenAIFunctionsAgentOutputParser()

In [210]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
    }
    | template
    | llm
    | OpenAIFunctionsAgentOutputParser()

)

In [ ]:
agent_executor = 

In [173]:
query = "What's the income for TSLA?"
tools = get_tools(query)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [179]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [180]:
agent_executor.invoke({
    "input": "What is the income for TSLA in 2022?",
})



> Entering new AgentExecutor chain...


ValueError: Can only parse messages